In [41]:
# from utils.processing import Processing
from mne.preprocessing import ICA
import matplotlib.pyplot as plt

import mne
import pathlib
import numpy as np
from scipy.stats import pearsonr
from scipy.stats import wilcoxon

from utils.processing import Processing

import random
import timeit

In [42]:
data = {}
for filename in pathlib.Path("./lee2019-artifacts/").glob("*SSVEP_train-raw*.fif"):
    data[filename] = mne.io.read_raw(filename, preload=True, verbose=False).pick(
        ["O1", "O2", "Fp1", "Fp2"]
    )

In [43]:
len(data)

54

In [44]:
import scipy.signal as signal


def hilbert_tranform(data: np.ndarray):
    """Computes analytic signal using Hilbert transform

    Args:
        data (np.ndarray): Data to compute analytic signal from
    """

    assert (
        data[0].shape[0] == data[1].shape[0]
    ), "Two data streams should have the same number of trials."
    data = np.array(data)

    # Hilbert transform
    complex_signal = []

    data_array = np.array([data[participant] for participant in range(2)])
    hilb = signal.hilbert(data_array)
    complex_signal.append(hilb)

    complex_signal = np.moveaxis(np.array(complex_signal), [0], [3])

    return complex_signal

In [45]:
from hypyp import analyses


def calculate_sync(epochs: mne.Epochs, parameter: str):
    """Calculates synchronization value

    Args:
        epochs (mne.Epochs): Epoched EEG data
        parameter (str): Synchronization parameter
    """

    values = hilbert_tranform(data=np.array(epochs))
    result = analyses.compute_sync(values, parameter, epochs_average=True)

    inter_values = result[:, 0:2, 2:4]

    AM = np.mean(inter_values)
    GM = np.prod(inter_values) ** (1 / 4)

    inter_sync = np.round(((AM + GM) / 2), 2)

    return inter_sync

In [46]:
random.seed(42)
sampled_data = random.sample(range(1, len(data)), 6)
sampled_data

[41, 8, 2, 48, 18, 16]

In [47]:
modes = ["ica", 
        #  "asr", "bilstm"
         ]

tmin = 1
tmax = 3
channels = ["O1", "O2"]
reg = "^((?!__).)*$"

cohs = []

In [48]:
def synchronization(subj1, subj2):
    events_subj1, names = mne.events_from_annotations(subj1, regexp=reg)
    events_subj1 = mne.pick_events(events_subj1)

    events_subj2, names = mne.events_from_annotations(subj2, regexp=reg)
    events_subj2 = mne.pick_events(events_subj2)

    for mode in modes:
        start = timeit.default_timer()
        subj1_clean = Processing(raw=subj1, mode=mode)
        subj2_clean = Processing(raw=subj2, mode=mode)

        subj1_epochs = mne.Epochs(
            subj1_clean.reconstructed,
            events=events_subj1,
            event_id=names,
            tmin=tmin,
            tmax=tmax,
            baseline=None,
            preload=True,
            verbose=False,
            picks=channels,
        )

        subj2_epochs = mne.Epochs(
            subj2_clean.reconstructed,
            events=events_subj2,
            event_id=names,
            tmin=tmin,
            tmax=tmax,
            baseline=None,
            preload=True,
            verbose=False,
            picks=channels,
        )

        coh = calculate_sync(
            epochs=[subj1_epochs, subj2_epochs], parameter="coh"
        )

        cohs.append(coh)

        stop = timeit.default_timer()

        print(
            f"Results when using {mode} mode:\n| {coh} | time = {np.round(stop-start)}\n"
        )

Group 1

In [50]:
_, subj1 = list(data.items())[sampled_data[0]]
_, subj2 = list(data.items())[sampled_data[1]]

synchronization(subj1, subj2)

Results when using ica mode:
| 0.17 | time = 8.0



Group 2

In [51]:
_, subj1 = list(data.items())[sampled_data[2]]
_, subj2 = list(data.items())[sampled_data[3]]

synchronization(subj1, subj2)

Results when using ica mode:
| 0.13 | time = 9.0



Group 3

In [53]:
_, subj1 = list(data.items())[sampled_data[4]]
_, subj2 = list(data.items())[sampled_data[5]]

synchronization(subj1, subj2)

Results when using ica mode:
| 0.15 | time = 8.0



In [64]:
# Mean and standard deviation for coherence and PLV
mean_coherence = np.round(np.mean(cohs), 2)
std_coherence = np.round(np.std(cohs), 2)

print(f"Coh value = {mean_coherence} +- {std_coherence}")


Coh value = 0.15 +- 0.02


In [56]:
cohs

[0.17, 0.13, 0.15]

In [70]:
import numpy as np
from scipy.stats import kruskal
from scipy.signal import coherence

# Example coherence values from actual subjects
# Assuming coherence values are stored in a list of lists, one for each pair
actual_coh_values = [0.2, 0.3, 0.25]  # Placeholder values

# Generate random signals and calculate coherence for each pair equivalent
random_coh_values = []
for _ in range(len(actual_coh_values)):
    # Generate random data
    random_signal1 = np.random.normal(0, 1, 1000)
    random_signal2 = np.random.normal(0, 1, 1000)

    # Calculate coherence
    _, coh = coherence(random_signal1, random_signal2, fs=256, nperseg=256)
    random_coh_values.append(np.mean(coh))


random_coh_values

[0.1635711811460089, 0.1597643563119058, 0.15895143360769973]

In [71]:
icas = [0.17, 0.13, 0.15]
asrs = [0.16, 0.14, 0.15]
bilstms = [0.3, 0.29, 0.21]

In [72]:
# Conduct Kruskal-Wallis H-test
stat, p_value = kruskal(asrs, random_coh_values)

print(f"Kruskal-Wallis H-test Statistic: {stat}")
print(f"P-value: {p_value}")

Kruskal-Wallis H-test Statistic: 1.1904761904761898
P-value: 0.27523352407483126
